## Import libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## 0.Read data

In [2]:
df_total_payments = pd.read_csv("../Datasets/clean_total_payments.csv")
df_purchase_orders = pd.read_csv("../Datasets/clean_purchase_orders.csv")
df_customer = pd.read_csv("../Datasets/clean_customer.csv")
df_employees = pd.read_csv("../Datasets/clean_employees.csv")
df_inventory = pd.read_csv("../Datasets/clean_inventory.csv")
df_order_details = pd.read_csv("../Datasets/clean_order_details.csv")
df_orders = pd.read_csv("../Datasets/clean_orders.csv")
df_products = pd.read_csv("../Datasets/clean_products.csv")